## scraping wayback machine

In [6]:
# imports
import sys
import requests as rq
from bs4 import BeautifulSoup as bs
from time import sleep
from time import time
from random import randint
from warnings import warn
import json
import pandas as pd
import numpy as np

In [7]:
# compile list of urls w/ wayback server CDX API
## http inlet that queries data from wayback machine 
### server responds to GET queries and outputs as JSON array

In [8]:
# url anatomy
# 'http://web.archive.org/cdx/search/cdx?url=nbcnews.com/politics&collapse=digest&from=20190401&to=20190431&output=json'
## url= ____ from= to=  output=json

## Set up CDX API query
to-do: 
    - get different time window for request
    - only pulled oct 14 2008?
NOTE: html has updated from 2002-2008 to include ids

In [9]:
## NOTE songs are listed from most recently played at top
# display order is not same as table order

In [10]:
## now-playing for 2002-2008
# - only shows songs from 7am-9am? add query to url for all time?

url = "http://web.archive.org/cdx/search/cdx?url=http://955thebeat.com/includes/nowplaying/lsp/&from=20020101&to=20081231&output=json"

# get request to the API for the text
urls = rq.get(url).text
# parses json text from urls
parse_url = json.loads(urls)

## compile url list- get timestamp and original columns from urls
url_list = []
for i in range(1, len(parse_url)):
    orig_url = parse_url[i][2]
    tstamp = parse_url[i][1]
    waylink = tstamp+'/'+orig_url
    url_list.append(waylink)

# compiles final url pattern
for url in url_list:
    final_url = 'https://web.archive.org/web/'+url

In [11]:
# final url list is only last item, need to include the rest of the code in this for loop
# when more urls are parsed, will need to update parsing code to include time/date data

## Parse html with BeautifulSoup

In [12]:
# open page to soupify
req = rq.get(final_url).text

# parse and store in soup
soup = bs(req,'html.parser')

In [13]:
# inspect page to look for id tags or other attributes -table with songs

In [14]:
## move through parse tree with tag.contents and string
# contents: ordered list of Tag and NavigableString objects in a page element
# if tag has only one child node and it is a string:
## tag.string or tag.contents[0]
# .children    .next

In [15]:
song_rows = soup.find_all("tr", {"height": 32})
songs = [i.find_all("td")[1].text.split("\n") for i in song_rows]

# clean up output with update and map/filter once df has been madeb

In [16]:
df = pd.DataFrame(songs, columns = ['title', 'artist'])
df.head()

,title,artist
0,Girlfriend,Bow Wow & Omarion - tour dates
1,Walk It Out,Unk & Outkast
2,Cyclone,BABY BASH/T-PAIN
3,BED,J. HOLIDAY - tou...
4,Temperature,Sean Paul


## cleaning the df

In [17]:
# remove " - tour dates "
df['artist'] = df['artist'].str.replace(" - tour dates", "")

In [18]:
# strip extra white text from artist
df['artist'] = df['artist'].str.strip()

In [19]:
df.head()

,title,artist
0,Girlfriend,Bow Wow & Omarion
1,Walk It Out,Unk & Outkast
2,Cyclone,BABY BASH/T-PAIN
3,BED,J. HOLIDAY
4,Temperature,Sean Paul


In [21]:
df.to_csv('./data/102008sample.csv', index=False)